#### **Задание 1 (0.5 поинтов для домашнего задания)**
1. Преобразуйте с помощью $tf-idf$ на все вектора из вашего сформированного корпуса (любого, создайте на выбор либо воспользуйтесь любыми текстами из датасета bbc по определенной тематике) и создайте новый индекс
2. Преобразуйте вектор запроса с помощью $tf-idf$
3. Как вычисляются $idf$ веса в векторе запроса?
4. Повторите поиск по запросу после $tf-idf$  преобразования

In [165]:
import re
from gensim import models, corpora, similarities
from nltk.corpus import stopwords
from collections import defaultdict

from pprint import pprint

##### Читаем документ препроцессим его, удаляя все лишнее

In [166]:
text = ''
for i in range(1, 417):
    file = open('politics/'f'{i:03}''.txt', 'r')
    text += file.read()
    file.close()
text = text.replace('\n\n', '\n')
text_list = text.split('\n')[:-1]

In [167]:
for i in range(len(text_list)):
    text_list[i] = text_list[i].lower()
    text_list[i] = re.sub('[^a-z ]', ' ', text_list[i])
    text_list[i] = re.sub('\s+', ' ', text_list[i])

In [168]:
stopwords = set(stopwords.words('english'))

texts = [[word for word in text_line.split() if word not in stopwords] for text_line in text_list]

##### Формируем словарь для данного текста и векторный корпус. После применяем $tf-idf$ преобразование на корпус текста

In [169]:
dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]

tfidf = models.TfidfModel(corpus)

##### Формируем запрос, а также векторное его представление и $tf-idf$ представление

In [170]:
q = "human computer interaction"
vec = dictionary.doc2bow(q.lower().split())
print(vec)
print(tfidf[vec])
vec_tf = tfidf[vec]

[(1460, 1), (5610, 1)]
[(1460, 0.45049967900009263), (5610, 0.892776589758498)]


In [171]:
index = similarities.MatrixSimilarity(corpus)
index_tf = similarities.MatrixSimilarity(tfidf[corpus])

##### Ищем максимально похожие строки в документе

In [172]:
sims = index[vec]
sims_tf = index_tf[vec_tf]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims_tf = sorted(enumerate(sims_tf), key=lambda item: -item[1])
print("Q:", q)

for i in sims[:5]:
    print('doc', i[0], text_list[i[0]], i[1])

print('-'*340)

for i in sims_tf[:5]:
    print('doc', i[0], text_list[i[0]], i[1])

Q: human computer interaction
doc 1711 ms khan highlighted detentions without trial including those at the us camp at guantanamo bay in cuba and the abuse of prisoners as evidence of increasing human rights problems what s a new challenge is the way in which this age old debate on security and human rights has been translated into the language of war she said by using the language of war human rights are being sidelined because we know human rights do not apply in times of war ms khan said such breaches were infectious and were now seen in almost very major country in the world the human rights movement faces a crisis of faith in the value of human rights she said that was accompanied by a crisis of governance where the united nations system did not seem able to hold countries to account  0.3407771
doc 2484 liberal democrat spokesman mark oaten also backed use of wire tap evidence he said the standard of proof for the new powers would have to be very high indeed and he asked whether mi

##### $idf$ формируется как натуральный логарифм для отношения общего количества документов (строк) к тому количеству документов (строк), в которых есть данное слово

#### **Задание 2 (0.5 поинтов для домашнего задания)**
1. Возьмите любую одну новостную статью из датасета Bbс (в папке data) по любой тематике.
2. Проведите препроцессинг текстового документа и разбейте статью на предложения.
3. Рассчитайте TF-IDF вектора из списка ваших предложений (то есть вместо документа будут предлоежния).
4. Рассчитайте метрику (например среднее значение ненулевых элементов в TF-IDF векторе)
5. Отсортируйте каждое предложение по метрике (т.е. постройте индекс)
6. Выведите топ 5 предложений, которые и будут финальным решением Extract-Based Summarization подхода.

In [173]:
file = open('politics/001.txt', 'r')
article = file.read()
file.close()
article = article.split('.')[:-1]

In [174]:
for i in range(len(article)):
    article[i] = article[i].lower()
    article[i] = re.sub('[^a-z ]', ' ', article[i])
    article[i] = re.sub('\s+', ' ', article[i])

In [175]:
article_words = [[word for word in subarticle.split() if word not in stopwords] for subarticle in article]

In [176]:
dictionary_article = corpora.Dictionary(article_words)

corpus_article = [dictionary_article.doc2bow(text) for text in article_words]

tfidf_article = models.TfidfModel(corpus_article)

In [177]:
print(tfidf_article[corpus_article[2]])

[(6, 0.18384201320076796), (15, 0.337535675630588), (16, 0.337535675630588), (17, 0.2157363298557076), (18, 0.337535675630588), (19, 0.13888949864079758), (20, 0.2157363298557076), (21, 0.2157363298557076), (22, 0.2157363298557076), (23, 0.337535675630588), (24, 0.337535675630588), (25, 0.337535675630588), (26, 0.27777899728159516)]


In [178]:
extract_based_summarization = []
for i in tfidf_article[corpus_article]:
    line_av = 0
    for c in i:
        line_av += c[1]
    extract_based_summarization.append(line_av/len(i))

In [179]:
sorted_article = [x for _, x in sorted(zip(extract_based_summarization, article))]

In [180]:
sorted_article[:5]

[' she said ministers would consult on other proposals that could see fathers being allowed to take some of their partner s maternity pay or leave period or extending the rights of flexible working to carers or parents of older children',
 ' the shadow secretary of state for the family theresa may said these plans were announced by gordon brown in his pre budget review in december and tony blair is now recycling it in his desperate bid to win back women voters',
 ' while the majority of any salary costs may be covered by the government s statutory pay recruitment costs advertising costs retraining costs and the strain on the company will not be he said',
 ' liberal democrat spokeswoman for women sandra gidley said while mothers would welcome any extra maternity pay the liberal democrats feel this money is being misdirected',
 ' she said her party would boost maternity pay in the first six months to allow more women to stay at home in that time']